In [1]:
''' DECISION TREE ALGORITHM FOR SEP CLASSIFICATION '''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

from datetime import datetime,timedelta

from pandas.plotting import scatter_matrix

from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, recall_score, matthews_corrcoef
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, roc_curve, brier_score_loss, precision_score
from sklearn.metrics import brier_score_loss as bsl
from sklearn.metrics import mean_squared_error as mse
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight
#import scikitplot as skplt


from scipy.sparse import csr_matrix, find


In [2]:
''' read in the Balch SEP event list '''

sepdata = pd.read_excel("SPEall.v7p.xls")

#read in the Balch SEP event list
old_data = pd.read_excel("ctrlevents.v8p.xls")

#read in the student event list
old_data = pd.read_excel("ControlEvents_student.xls")
old_data.TypeII = (old_data.TypeII.str.lower() == "yes").astype(int)
old_data.TypeIV = (old_data.TypeIV.str.lower() == "yes").astype(int)

#adding y label column indicating positive and negative SEP events - SEP events have Association = ProtonFlare
#old_data['sep'] = (old_data.Association == "ProtonFlare").astype(int)
old_data['sep'] = old_data.Association.str.contains('^Proton').astype(int)


new_data = pd.read_csv('new_sep.csv')

In [3]:
feature_data_old = old_data[['FlrOnset','Flrmaxtime','FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV','optlocation','tchianti','emchianti','FlrIntFlux','sep']]
feature_data_new = new_data[['FlrOnset','Flrmaxtime','FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV','optlocation','tchianti','emchianti','FlrIntFlux','sep']]

In [4]:
data = pd.concat([feature_data_old,feature_data_new])


In [5]:

# Remove rows where optlocation = nan
old_data = old_data[old_data.optlocation.astype('str') != 'nan']
#shuffle the events so they are not organized 
old_data = shuffle(old_data)
#save the shuffled dataframe -- commented out to prevent resaving
old_data.to_csv("AllEvtsShuffled_student_PB.csv")

# Remove rows where optlocation = nan
data = data[data.optlocation.astype('str') != 'nan']
#shuffle the events so they are not organized 
data = shuffle(data)
#save the shuffled dataframe -- commented out to prevent resaving
data.to_csv("AllEvtsShuffled_PB.csv")

In [6]:
class BalchPaperFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        """Class to create original 4 features from Balch 2008"""
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ['FlrPeakFlux','FlrIntFlux','TypeII','TypeIV']
    
    def transform(self, examples):
                
        #Choose the orginal 4 Balch 2008 features (5 if you include both type II and type Iv as separate features)
        #X = examples[['FlrPeakFlux','FlrIntFlux','Type24']]
        X = examples[['FlrPeakFlux','FlrIntFlux2','TypeII','TypeIV']]
        #temp = np.logical_or(examples['TypeII'],examples['TypeIV'])
        #X['TypeII_IV'] = temp
        

        return(X)

In [7]:
class RawFeatures(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        """Ogher features from the Event List that don't need to be manipulated before being included in model"""
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ['FlrIntFlux','tchianti','emchianti']
    
    def transform(self, examples):
                
        #Choose the orginal 4 Balch 2008 features (5 if you include both type II and type Iv as separate features)
        X = examples[['FlrIntFlux','tchianti','emchianti']]

        return(X)

In [8]:
class FlareTime2Peak(BaseEstimator, TransformerMixin):
    
    """Class to create feature with the time between flare onset to flare max time"""
    
    def __init__(self):
        
        return None
    
    def fit(self, examples):
        # return self and nothing else 
        return self
    
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
        
        return ["FlTim2Pk"]
    
    def transform(self, examples):
        
        # Initiaize matrix 
        X = np.zeros((len(examples),1))           

        #time between flare max and flare onset
        X[:,0] = np.asarray([(mx - on).seconds for mx, on in zip(examples.Flrmaxtime, examples.FlrOnset)])
        
        return(X)

In [9]:
class LocationFeatures(BaseEstimator, TransformerMixin):
   
    """Class to create feature with the time between flare onset to flare max time"""
   
    def __init__(self):
       
        return None
   
    def fit(self, examples):
        # return self and nothing else 
        return self
   
    def get_feature_names(self):
        """Array mapping from feature integer indices to feature name"""
       
        return ['Loc']
   
    def transform(self, examples):
       
        # Initiaize matrix 
        X = np.zeros((len(examples),2))  
       
        #time between flare max and flare onset
        #X[:,0] = np.asarray([str(x)[1:3] for x in examples.optlocation])   #north - south
        #X[:,1] = np.asarray([str(x)[4::] for x in examples.optlocation])   #north - south
       
        for i,loc in enumerate(examples.optlocation):
            lat = str(loc)[1:3]
            if str(loc)[0] == 's' or str(loc)[0] == 'S':
                X[i,0] = -int(lat)
            else:
                X[i,0] = (lat)

            #west-east
            long = str(loc)[4::]
            if str(loc)[3] == 'w' or str(loc)[3] == 'W':
                X[i,1] = -int(long)
            else:
                X[i,1] = int(long)

        return(X)

In [10]:
class SEPClass(BaseEstimator, ClassifierMixin):
    def __init__(self, estimator, folds = 6, threshold=0.5):        # <--- other keywords to be used by Feature Union go here
        
        """Class to fit and train Logistic Regression algorithm for SEP forecasting
        
        Input keywords:
        
        folds:        Number of cross validation folds to use
        threshold:    Decision Boundary threshold
        
        """
        
        self.estimator = estimator     #estimator to use for classification e.g. LogReg or SVM
        self.folds = folds             #cross validation folds for estimator
        self.threshold = threshold     #decision boundary threshold
    
        #Set up the Feature union to combine Feature creating classes
        self.allmyfeatures = FeatureUnion([
            ("BalchFeat", BalchPaperFeatures()),
            ("RawFeat", RawFeatures()),
            ("LocFeatures", LocationFeatures())#,
            #("FlareTime2Peak", FlareTime2Peak())   
        ])
    
    def build_train_features(self, examples):
        """
        Method to take in training text features and do further feature engineering 
        Most of the work in this homework will go here, or in similar functions  
        :param examples: currently just a list of forum posts  
        """  
        
        ##convert columns of time from string to datetime -- MOVE TO SEPARATE CLEANING FUNCTION
        examples.FlrOnset = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.FlrOnset)
        examples.Flrmaxtime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.Flrmaxtime)
        #examples.Flrendtime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.Flrendtime)
        #examples.FlrHpTime = pd.Series(datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%f") for t in examples.FlrHpTime)
        
        ##convert Type II and Type IV "yes/no" to binary
        #print(examples.TypeII)
        #examples.TypeII = (examples.TypeII.str.lower() == "yes").astype(int)
        #examples.TypeIV = (examples.TypeIV.str.lower() == "yes").astype(int)
        #examples['Type24'] = np.logical_or(examples.TypeII, examples.TypeIV)
        
        return self.allmyfeatures.fit_transform(examples)
        #normalize data here?

    def get_test_features(self, examples):
        """
        Method to take in test text features and transform the same way as train features 
        :param examples: currently just a list of forum posts  
        """
    
        return self.allmyfeatures.transform(examples)

    def show_topX(self, num=3):
        """
        prints the top num features for the positive class and the 
        top 10 features for the negative class. 
        """
        feature_names = np.asarray([x.split("__")[1] for x in self.allmyfeatures.get_feature_names()])
        topX = np.argsort(self.estimator.coef_[0])[-num:]
        bottomX = np.argsort(self.estimator.coef_[0])[:num]
        
        print(feature_names)
        
        print("\nTop 3 features for Pos and Neg\n-------------------------")
        for fn in np.arange(1,num):
            print("Pos %i: %s %f" % (fn, feature_names[topX[-fn]], self.estimator.coef_[0,topX[-fn]]))
        for fn in np.arange(0,num-1):
            print("Neg %i: %s %f" % (fn, feature_names[bottomX[fn]], self.estimator.coef_[0,bottomX[fn]]))
            
      
    def show_misclassified(self):     

        """
        Method to show the misclassified examples i.e. False Positives and False negatives 
        """
        
        #get all the feature names
        #words = feat.allmyfeatures.get_feature_names() #####
        words = self.allmyfeatures.get_feature_names() #####
        
        # False positives
        print("\nSome misclassified examples:")
        falsepos = np.where((self.train_pred != self.y_train) & (self.train_pred == 1))[0]   #all false pos example rows
        print("\nPredicted SEP but labeled AllClear (False Pos) \n------------------------- ")

        for i in range(len(falsepos[0:10])):         #loop through falsepos examples
            weights_falsepos = []
            x = find(feat.X_train[falsepos[i]])      #find which features are used for this example
            for ii in x[1]:                          #from sparse matrix get column indices corresponding to features
                weights_falsepos.append((words[ii].split('__')[1], self.estimator.coef_[0,ii]))      #get the word and weight

            print("label: %i, prediction %i, Neg Prob: %f, Pos Prob: %f, Ex No.: %i,  example: %s " % \
                (self.y_train[falsepos[i]], self.train_pred[falsepos[i]] , self.train_pred_prob[falsepos[i]][0], \
                     self.train_pred_prob[falsepos[i]][1], falsepos[i], self.clean_examples[falsepos[i]]))
            for j in weights_falsepos:
                print(j)
                
        # False Negatives
        falseneg = np.where((self.train_pred != self.y_train) & (self.train_pred == 0))[0]
        print("\nPredicted AllClear but labeled SEP (False Neg) \n-------------------------")

        for i in range(len(falseneg[0:10])):
            weights_falseneg = []
            x = find(feat.X_train[falseneg[r]])
            for ii in x[1]:
                weights_falseneg.append((words[ii].split('__')[1], self.estimator.coef_[0,ii])) 

            print("label: %i, prediction %i, Neg Prob: %f, Pos Prob: %f, Ex No.: %i, \nexample: %s " % \
                (self.y_train[falseneg[i]], self.train_pred[falseneg[i]], self.train_pred_prob[falseneg[i]][0], \
                 self.train_pred_prob[falseneg[i]][1], falseneg[i], self.clean_examples[falseneg[i]]))
            for j in weights_falseneg:
                print(j)
        
        print("\n\n")
        print(words)
       
    def show_report(self):
        
        """Method to show a report card of the model fit """
        
        # Add confusion Matrix
        tn, fp, fn, tp=confusion_matrix(self.y_train, self.y_train_pred).ravel()
        print("True Pos: %i, True Neg: %i, False Pos: %i,. False Neg: %i\n" % (tp,tn,fp,fn))

        #calculate the mean square error
        mserr = mse(self.y_train, self.y_train_pred)
        print("MSE: %.4f" % mserr)

        #calculate the Brier score - is this the same as the MSE? And QR referenced in Balch paper?
        bsloss = bsl(self.y_train, self.y_train_pred_prob[:,1])
        print("BSL: %.4f" % bsloss)

        #Occurance rate = #SEPS / #events
        occ_rate = self.y_train.sum()/len(self.y_train)

        #Reference score of predicting all negative class
        #QR_star = 0.0324 Balch
        QR_star =  mse(self.y_train, np.zeros(len(self.y_train)))
        print("RefQuadScore: %.4f\n" % QR_star)

        #assume the QR is the same as the MSE - as stated later in Balch
        QR = mserr
        #QR = 0.0250 Balch

        #skill score 
        SS = (QR_star - QR)/QR_star
        print("SS: %.4f" % SS)

        FAR = fp/(tp + fp)
        POD = tp/(tp + fn)
        TSS = tp / (tp+fp+fn)
        E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / self.X_train.shape[0]
        HSS = (tp + tn - E)/(self.X_train.shape[0] - E)

        print("FAR: %.4f" % FAR)
        print("POD: %.4f" % POD)
        print("TSS: %.4f" % TSS)
        print("HSS: %.4f" % HSS)
        
        ##ROC metrics
        fpr, tpr, thresh = roc_curve(self.y_train, self.y_train_pred, drop_intermediate=False)

    def roc_curve(self):
        
        """plot and ROC curve"""
        
        # Initial implementation of ROC plot applied to training set
        skplt.metrics.plot_roc_curve(self.y_train, self.y_train_probas)
        
    def score(self, X, y):

        """find the accuracy score given the training data and labels"""
        #print("...In Score...")
        #print("threshold:", self.threshold)
        
        y_pred = (self.estimator.predict(X) > self.threshold).astype(int)
        
        #tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
        #return tp/(tp+fp+fn)
        
        return accuracy_score(y, y_pred)
        
    def fit(self, X, y):    #, random_state=1234):
        """
        Method to read in training data from file, and 
        train Logistic Regression classifier. 
        
        :param random_state: seed for random number generator 
        """
        
        from sklearn.linear_model import LogisticRegression 
        
        # load data 
        #self.dfTrain = pd.read_csv("AllEvtsShuffled.csv")
                      
        # get training features and labels 
        #self.X_train = self.build_train_features(self.dfTrain)    #CHANGE
        #self.y_train = np.array(self.dfTrain.sep, dtype=int)
        
        self.X_train = X 
        self.y_train = y 
        
        #print the shape of the features
        print("Shape of the Features: Num examples x Num Features")
        print(self.X_train.shape)
        #print("examples...:", self.X_train[0:10])

        #self.logreg.fit(self.X_train, self.y_train)
        self.estimator.fit(self.X_train, self.y_train)
        
        # make predictions on training data 
        self.y_train_pred = self.estimator.predict(self.X_train)

        #return the LogReg probabilities used to classify each example  
        self.y_train_pred_prob = self.estimator.predict_proba(self.X_train)
        
        print("\nTRAINING SET\n-------------------------------------------------------------")
        #self.score(self.train_pred, self.y_train)
             
        #cross validation
         
        metric1 = 'recall'
        metric2 = 'precision'
    
        metric3s = 'matthews correlation coefficient' 
        metric3 = make_scorer(matthews_corrcoef)
        
        metric4s = "brier score" 
        metric4 = make_scorer(brier_score_loss)
        
        #print("\nCross Validation Metric Scores (", metric4s, ")\n-------------------------")
        #scores = cross_val_score(self.estimator, self.X_train, self.y_train, cv=self.folds,scoring=metric4)
        #print(scores)
        #print("\nMean Accuracy in Cross-Validation = %.3f \n" % scores.mean())
        
        print("\nCross Validation Metric Scores\n-----------------------------------\n")
        rec_score = cross_val_score(self.estimator, self.X_train, self.y_train, cv=self.folds,scoring=metric1)
        prec_score = cross_val_score(self.estimator, self.X_train, self.y_train, cv=self.folds,scoring=metric2)
        mattco_score = cross_val_score(self.estimator, self.X_train, self.y_train, cv=self.folds,scoring=metric3)
        brier_score = cross_val_score(self.estimator, self.X_train, self.y_train, cv=self.folds,scoring=metric4)
        
        print(metric1,'\n')
        print(rec_score,'\n-------------------------\n')
        
        print(metric2,'\n')
        print(prec_score,'\n-------------------------\n')
        
        print(metric3s,'\n')
        print(mattco_score,'\n-------------------------\n')
        
        print(metric4s,'\n')
        print(brier_score,'\n-------------------------\n')
        
        #cross validation
        
        #cross validation
        #print("\nCross Validation Accuracy Scores (cross_val_predict)\n-------------------------")
        #self.y_pred = cross_val_predict(self.estimator, self.X_train, self.y_train, cv=self.folds)
        #print(self.score(self.y_pred, self.y_train))
        
        #print("\nCross Confusion Matrix\n-------------------------")
        #self.conf_mat = confusion_matrix(self.y_train,self.y_pred)
        #tn, fp, fn, tp = self.conf_mat.ravel()
        #print("True Pos: %i, True Neg: %i, False Pos: %i,. False Neg: %i" % (tp,tn,fp,fn))
        
        #training set score (no cross-validation)
        print('\nTraining Set (no cross-validation) Score \n-----------------------------------\n')
        
        rec1_score = recall_score(self.y_train,self.y_train_pred)
        prec1_score = precision_score(self.y_train,self.y_train_pred)
        mattco1_score = matthews_corrcoef(self.y_train,self.y_train_pred)
        
        print(metric1,'\n')
        print(rec1_score,'\n-------------------------\n')
        
        print(metric2,'\n')
        print(prec1_score,'\n-------------------------\n')
        
        print(metric3s,'\n')
        print(mattco1_score,'\n-------------------------\n')
        
        #brier score is already on report card
        print('Briar Score (BSL) on report card\n')
        print('-------------------------------------------------------------\n')
        
        #print report card
        print("\nReport Card:\n----------------------------------------\n")
        print(sep.show_report())
        
    
    def predict(X):
        
        """
        Return predicted labels for exmaples X. 
 
        #### CURRENTLY THIS FUNCTION ISN'T USED - but could be called if we need predicted y vals 
        #### independently to the score function
 
        """
        print("INside predict....")
        print("Thresh: ", self.threshold)
        return (self.estimator.predict(X) > self.threshold).astype(int)
        
    def model_predict(self):
        """
        Method to read in test data from file, make predictions
        using trained model, and dump results to file 
        
        #### CURRENTLY THIS FUNCTION ISN'T USED - leftover from FeatEngr homework but we might need to
        #### to test on the holy grail test set
        """
        
        # read in test data 
        dfTest  = pd.read_csv("../data/spoilers/test.csv")
        
        # featurize test data 
        self.X_test = self.get_test_features(list(dfTest["sentence"]))
        
        # make predictions on test data 
        pred = self.estimator.predict(self.X_test)
        
        # dump predictions to file for submission to Kaggle  
        #pd.DataFrame({"spoiler": np.array(pred, dtype=bool)}).to_csv("prediction.csv", index=True, index_label="Id")
        
                

In [11]:
# load data - for the gridsearcv the data needs to be loaded outside of the class
dfTrain = pd.read_csv("AllEvtsShuffled_student_PB.csv")          

#create training and test set
sep = SEPClass(LogisticRegression())

#Turn off features that aren't fully fleshed out or don't work
#sep.allmyfeatures.set_params(RawFeat=None)

# get training features and labels 
X_total = sep.build_train_features(dfTrain)    #CHANGE
y_total = np.array(dfTrain.sep, dtype=int)

#split dataset into training and test
X_train, X_test, y_train, y_test = train_test_split(X_total,y_total,test_size=0.2,random_state=1230,stratify=y_total)

#standardization
#X_std = preprocessing.scale(X_train)

In [17]:
''' DECISION TREES AND BOOSTING '''

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier as trees

metric3 = make_scorer(matthews_corrcoef)

def tss_score(y, y_pred):
    tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
    tss = tp / (tp + fp + fn)
    return tss

def hss_score(y, y_pred):
    tn, fp, fn, tp=confusion_matrix(y, y_pred).ravel()
    E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / X_train.shape[0]
    HSS = (tp + tn - E)/(X_train.shape[0] - E)
    return HSS

metric5 = make_scorer(tss_score)
metric6 = make_scorer(hss_score)

dtc_keywords = {'max_depth':1, 'max_features':None,'random_state':1230,'min_samples_split':2,'min_samples_leaf':1}
clf_keywords = {'random_state':1230}
        
dtc = trees(**dtc_keywords)
clf = AdaBoostClassifier(dtc,**clf_keywords)

dtc_params = {'criterion':['gini','entropy']}
clf_params = {'n_estimators':range(50,92,2),'learning_rate':np.arange(1.6,0.1)}
#50-100, 0.1-1

dtc_search = GridSearchCV(dtc,dtc_params,scoring = metric5)
dtc_search.fit(X_train,y_train)
dtc_keywords.update(dtc_search.best_params_)
print(dtc_search.best_params_)

dtc_keywords.update(dtc_search.best_params_)
dtc = trees(**dtc_keywords)
clf = AdaBoostClassifier(dtc,**clf_keywords)

clf_search = GridSearchCV(clf,clf_params, scoring = metric5)
clf_search.fit(X_train,y_train)
clf_keywords.update(clf_search.best_params_)
print(clf_search.best_params_)

#print(clf_search.cv_results_)

{'criterion': 'gini'}
{'learning_rate': 1.6000000000000005, 'n_estimators': 72}


In [18]:
#train the model
dtc = trees(**dtc_keywords)
clf = AdaBoostClassifier(dtc,**clf_keywords)
clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1230,
            splitter='best'),
          learning_rate=1.6000000000000005, n_estimators=72,
          random_state=1230)

In [19]:
#train the model
#dtc = trees(max_depth = 1, max_features = None, random_state = 1230, min_samples_split = 2, min_samples_leaf = 1)
#clf = AdaBoostClassifier(dtc,random_state = 1230,learning_rate = 0.25, n_estimators = 200 )
#clf.fit(X_train,y_train)

In [20]:
#Decision Tree Scores
#cross validation
folds = 6       
    
metric1 = 'recall'
metric2 = 'precision'
    
metric3s = 'matthews correlation coefficient' 
metric3 = make_scorer(matthews_corrcoef)

metric4s = "brier score" 
metric4 = make_scorer(brier_score_loss)

metric5s = 'true skill score'

metric6s = 'Heidke Skill Score'
        
        
rec_score = cross_val_score(clf, X_train, y_train, cv=folds,scoring=metric1)
prec_score = cross_val_score(clf, X_train, y_train, cv=folds,scoring=metric2)
mattco_score = cross_val_score(clf, X_train, y_train, cv=folds,scoring=metric3)
brier_score = cross_val_score(clf, X_train, y_train, cv=folds,scoring=metric4)
tss_score = cross_val_score(clf, X_train, y_train, cv=folds,scoring=metric5)
hss_score = cross_val_score(clf, X_train, y_train, cv=folds,scoring=metric6)

print(metric1,'\n')
print(rec_score,'\n-------------------------\n')
        
print(metric2,'\n')
print(prec_score,'\n-------------------------\n')
        
print(metric3s,'\n')
print(mattco_score,'\n-------------------------\n')
        
print(metric4s,'\n')
print(brier_score,'\n-------------------------\n')

print(metric5s,'\n')
print(tss_score,'\n-------------------------\n')

print(metric6s,'\n')
print(hss_score,'\n-------------------------\n')

#training set score (no cross-validation)
y_train_pred = clf.predict(X_train)
y_train_pred_prob = clf.predict_proba(X_train)
print('\nTraining Set (no cross-validation) Score \n-----------------------------------\n')
        
rec1_score = recall_score(y_train,y_train_pred)
prec1_score = precision_score(y_train,y_train_pred)
mattco1_score = matthews_corrcoef(y_train,y_train_pred)
#tss1_score = tss_score(y_train,y_train_pred)
        
print(metric1,'\n')
print(rec1_score,'\n-------------------------\n')
        
print(metric2,'\n')
print(prec1_score,'\n-------------------------\n')
        
print(metric3s,'\n')
print(mattco1_score,'\n-------------------------\n')

#print(metric5s,'\n')
#print(tss1_score,'\n-------------------------\n')



"""Method to show a report card of the model fit """

#calculate the mean square error
mserr = mse(y_train, y_train_pred)
print("MSE: %.4f" % mserr)

#calculate the Brier score - is this the same as the MSE? And QR referenced in Balch paper?
bsloss = bsl(y_train, y_train_pred_prob[:,1])
print("BSL: %.4f" % bsloss)

#Occurance rate = #SEPS / #events
occ_rate = y_train.sum()/len(y_train)

#Reference score of predicting all negative class
#QR_star = 0.0324 Balch
QR_star =  mse(y_train, np.zeros(len(y_train)))
print("RefQuadScore: %.4f\n" % QR_star)

#assume the QR is the same as the MSE - as stated later in Balch
QR = mserr
#QR = 0.0250 Balch​-

#skill score 
SS = (QR_star - QR)/QR_star
print("SS: %.4f" % SS)

tn, fp, fn, tp=confusion_matrix(y_train, y_train_pred).ravel()
FAR = fp/(tp + fp)
POD = tp/(tp + fn)
TSS = tp / (tp+fp+fn)
E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / X_train.shape[0]
HSS = (tp + tn - E)/(X_train.shape[0] - E)

print("FAR: %.4f" % FAR)
print("POD: %.4f" % POD)
print("TSS: %.4f" % TSS)
print("HSS: %.4f" % HSS)

##ROC metrics
fpr, tpr, thresh = roc_curve(y_train, y_train_pred, drop_intermediate=False)

##Confusion Matrix

print("True Pos: %i, True Neg: %i, False Pos: %i,. False Neg: %i\n" % (tp,tn,fp,fn))

print("\nConfusion Matrix\n-------------------------")
conf_mat = confusion_matrix(y_train,y_train_pred) 
print(conf_mat)

recall 

[0.26315789 0.26315789 0.22222222 0.38888889 0.33333333 0.66666667] 
-------------------------

precision 

[0.5        0.625      0.4        0.63636364 0.35294118 0.63157895] 
-------------------------

matthews correlation coefficient 

[0.34618315 0.39225458 0.28013451 0.48409825 0.32044501 0.63612521] 
-------------------------

brier score 

[0.03591682 0.03219697 0.03795066 0.028463   0.04364326 0.02466793] 
-------------------------

true skill score 

[0.20833333 0.22727273 0.16666667 0.31818182 0.20689655 0.48      ] 
-------------------------

Heidke Skill Score 

[0.13835339 0.13868217 0.13751659 0.13918236 0.13684729 0.14017754] 
-------------------------


Training Set (no cross-validation) Score 
-----------------------------------

recall 

0.4909090909090909 
-------------------------

precision 

0.782608695652174 
-------------------------

matthews correlation coefficient 

0.6095543010316704 
-------------------------

MSE: 0.0224
BSL: 0.2293
RefQuadScore: 

In [21]:
#test set score (no cross-validation)
y_test_pred = clf.predict(X_test)
y_test_pred_prob = clf.predict_proba(X_test)
print('\nTraining Set (no cross-validation) Score \n-----------------------------------\n')
        
rec1_score = recall_score(y_test,y_test_pred)
prec1_score = precision_score(y_test,y_test_pred)
mattco1_score = matthews_corrcoef(y_test,y_test_pred)
#tss1_score = tss_score(y_train,y_train_pred)
        
print(metric1,'\n')
print(rec1_score,'\n-------------------------\n')
        
print(metric2,'\n')
print(prec1_score,'\n-------------------------\n')
        
print(metric3s,'\n')
print(mattco1_score,'\n-------------------------\n')

#print(metric5s,'\n')
#print(tss1_score,'\n-------------------------\n')



"""Method to show a report card of the model fit """

#calculate the mean square error
mserr = mse(y_test, y_test_pred)
print("MSE: %.4f" % mserr)

#calculate the Brier score - is this the same as the MSE? And QR referenced in Balch paper?
bsloss = bsl(y_test, y_test_pred_prob[:,1])
print("BSL: %.4f" % bsloss)

#Occurance rate = #SEPS / #events
occ_rate = y_test.sum()/len(y_test)

#Reference score of predicting all negative class
#QR_star = 0.0324 Balch
QR_star =  mse(y_test, np.zeros(len(y_test)))
print("RefQuadScore: %.4f\n" % QR_star)

#assume the QR is the same as the MSE - as stated later in Balch
QR = mserr
#QR = 0.0250 Balch​-

#skill score 
SS = (QR_star - QR)/QR_star
print("SS: %.4f" % SS)

tn, fp, fn, tp=confusion_matrix(y_test, y_test_pred).ravel()
FAR = fp/(tp + fp)
POD = tp/(tp + fn)
TSS = tp / (tp+fp+fn)
E = ((tp + fn)*(tp + fp) + (fp + tn)*(fn + tn)) / X_train.shape[0]
HSS = (tp + tn - E)/(X_train.shape[0] - E)

print("FAR: %.4f" % FAR)
print("POD: %.4f" % POD)
print("TSS: %.4f" % TSS)
print("HSS: %.4f" % HSS)

##ROC metrics
fpr, tpr, thresh = roc_curve(y_test, y_test_pred, drop_intermediate=False)

##Confusion Matrix

print("True Pos: %i, True Neg: %i, False Pos: %i,. False Neg: %i\n" % (tp,tn,fp,fn))

print("\nConfusion Matrix\n-------------------------")
conf_mat = confusion_matrix(y_test,y_test_pred) 
print(conf_mat)


Training Set (no cross-validation) Score 
-----------------------------------

recall 

0.39285714285714285 
-------------------------

precision 

0.4782608695652174 
-------------------------

matthews correlation coefficient 

0.41477551064581114 
-------------------------

MSE: 0.0366
BSL: 0.2293
RefQuadScore: 0.0354

SS: -0.0357
FAR: 0.5217
POD: 0.3929
TSS: 0.2750
HSS: 0.1937
True Pos: 11, True Neg: 752, False Pos: 12,. False Neg: 17


Confusion Matrix
-------------------------
[[752  12]
 [ 17  11]]


In [ ]:
from sklearn.metrics import roc_curve

#ROC
fpr, tpr, thresholds = roc_curve(y_train,y_train_pred_prob[:,1],pos_label = 1)

In [ ]:
threshold = 0.50
temp = y_train_pred_prob[:,1] > threshold
type(temp)
temp.astype('int')
temp2 = y_train + temp
tp = len(np.where(temp2 == 2)[0])

In [ ]:
#fp,fn,tss
fp = len(np.where(np.logical_and(temp == 1, y_train == 0))[0])
fn = len(np.where(np.logical_and(temp == 0, y_train == 1))[0])